# Challenge 4 study - Quantized Model with Conv Input Layer

## Import libraries and data, init virtual device

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, ReLU, Conv2D, Flatten, Dropout

from akida import Device, AKD1000
import cnn2snn

device = AKD1000()

2022-11-11 10:23:11.491770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 10:23:11.669317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-11 10:23:11.669350: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-11 10:23:11.712468: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 10:23:12.670897: W tensorflow/stream_executor/platform/de

## Data Preparation

In [2]:
coords_ch4 = np.load('coords_ch4.npy')
color = np.load('result_ch4.npy')

# Pack in a dataframe and scale for 8-bits quantization (first layer will be quantized on 8-bits)
data = pd.DataFrame(coords_ch4, columns = ['x', 'y'])
data = round(255*data).astype(np.uint8)


# Shuffle and split train/test
data['color'] = color
data_train = data.sample(frac=0.7, axis=0)
data_test = data.drop(data_train.index)

# Split x y
x_train = data_train.drop('color', axis=1)
y_train = data_train['color']
x_test = data_test.drop('color', axis=1)
y_test = data_test['color']


# Attempt to encode coordinates as an image: put the x value as grey scale on the left, then y on the right
left = np.empty((6, 3))
right = np.empty((6, 3))

x_train_coded = np.empty((7000, 6, 6))
for i, p in enumerate(x_train.to_numpy()):
    left.fill(p[0])
    right.fill(p[1])
    x_train_coded[i] = np.concatenate((left, right), axis=1)
x_train_coded = x_train_coded.reshape(-1, 6, 6, 1).astype(np.uint8)

x_test_coded = np.empty((3000, 6, 6))
for i, p in enumerate(x_test.to_numpy()):
    left.fill(p[0])
    right.fill(p[1])
    x_test_coded[i] = np.concatenate((left, right), axis=1)
x_test_coded = x_test_coded.reshape(-1, 6, 6, 1).astype(np.uint8)

x_train_coded[0]

array([[[221],
        [221],
        [221],
        [210],
        [210],
        [210]],

       [[221],
        [221],
        [221],
        [210],
        [210],
        [210]],

       [[221],
        [221],
        [221],
        [210],
        [210],
        [210]],

       [[221],
        [221],
        [221],
        [210],
        [210],
        [210]],

       [[221],
        [221],
        [221],
        [210],
        [210],
        [210]],

       [[221],
        [221],
        [221],
        [210],
        [210],
        [210]]], dtype=uint8)

## Model creation with Keras

In [3]:
base = Sequential(name="Base_ch4")
base.add(Conv2D(1, (3, 3), strides=(3, 3), input_shape=(6, 6, 1), name='C2D'))
base.add(ReLU(name='relu0'))
base.add(Dropout(0.2))
base.add(Flatten())
base.add(Dense(1000, name='FC1'))
base.add(ReLU(name='relu1'))
base.add(Dense(500, name='FC2'))
base.add(ReLU(name='relu2'))
base.add(Dense(250, name='FC3'))
base.add(ReLU(name='relu3'))
base.add(Dense(100, name='FC4'))
base.add(ReLU(name='relu4'))
base.add(Dense(50, name='FC5'))
base.add(ReLU(name='relu5'))
base.add(Dense(1, name='FC6'))
base.add(Activation('sigmoid', name='sigmoid'))

#base.summary()

## Model quantization (4-bits, input: 8-bit)

In [4]:
qmodel = cnn2snn.quantize(base, weight_quantization=4, activ_quantization=4, input_weight_quantization=8)
qmodel.summary()

print('\nAkida Compatibility:', cnn2snn.check_model_compatibility(qmodel, input_is_image=True))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C2D (QuantizedConv2D)       (None, 2, 2, 1)           10        
                                                                 
 relu0 (QuantizedReLU)       (None, 2, 2, 1)           0         
                                                                 
 dropout (Dropout)           (None, 2, 2, 1)           0         
                                                                 
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 FC1 (QuantizedDense)        (None, 1000)              5000      
                                                                 
 relu1 (QuantizedReLU)       (None, 1000)              0         
                                                                 
 FC2 (QuantizedDense)        (None, 500)              

## Preview akida model

In [5]:
akmodel = cnn2snn.convert(qmodel, input_is_image=True)
akmodel.map(device)
akmodel.summary()

                   Model Summary                   
___________________________________________________
Input shape  Output shape  Sequences  Layers  NPs
[6, 6, 1]    [1, 1, 1]     1          7       6  
___________________________________________________

________________________________________________________
Layer (type)      Output shape  Kernel shape       NPs

====== HW/C2D-FC6 (Hardware) - size: 560560 bytes ======

C2D (InputConv.)  [2, 2, 1]     (3, 3, 1, 1)       N/A
________________________________________________________
FC1 (Fully.)      [1, 1, 1000]  (1, 1, 4, 1000)    1  
________________________________________________________
FC2 (Fully.)      [1, 1, 500]   (1, 1, 1000, 500)  1  
________________________________________________________
FC3 (Fully.)      [1, 1, 250]   (1, 1, 500, 250)   1  
________________________________________________________
FC4 (Fully.)      [1, 1, 100]   (1, 1, 250, 100)   1  
________________________________________________________
FC5 (Fully.)

## Train quantized model

In [6]:
qmodel.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
            )
qmodel.fit(x_train_coded, y_train, batch_size=64, epochs=600)
qmodel.evaluate(x_test_coded, y_test)

Epoch 1/600
110/110 [==============================] - 6s 27ms/step - loss: 0.7329 - accuracy: 0.5001
Epoch 2/600
110/110 [==============================] - 3s 27ms/step - loss: 0.7063 - accuracy: 0.5020
Epoch 3/600
110/110 [==============================] - 3s 27ms/step - loss: 0.7013 - accuracy: 0.5053
Epoch 4/600
110/110 [==============================] - 3s 27ms/step - loss: 0.7069 - accuracy: 0.4986
Epoch 5/600
110/110 [==============================] - 3s 27ms/step - loss: 0.7058 - accuracy: 0.4930
Epoch 6/600
110/110 [==============================] - 3s 28ms/step - loss: 0.7041 - accuracy: 0.4951
Epoch 7/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6993 - accuracy: 0.5131
Epoch 8/600
110/110 [==============================] - 3s 27ms/step - loss: 0.7005 - accuracy: 0.5054
Epoch 9/600
110/110 [==============================] - 3s 27ms/step - loss: 0.6992 - accuracy: 0.5074
Epoch 10/600
110/110 [==============================] - 3s 27ms/step - loss: 0.703

110/110 [==============================] - 4s 32ms/step - loss: 0.6918 - accuracy: 0.5143
Epoch 81/600
110/110 [==============================] - 3s 31ms/step - loss: 0.6918 - accuracy: 0.5184
Epoch 82/600
110/110 [==============================] - 3s 31ms/step - loss: 0.6909 - accuracy: 0.5234
Epoch 83/600
110/110 [==============================] - 4s 33ms/step - loss: 0.6924 - accuracy: 0.5130
Epoch 84/600
110/110 [==============================] - 3s 31ms/step - loss: 0.6912 - accuracy: 0.5141
Epoch 85/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6918 - accuracy: 0.5166
Epoch 86/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6916 - accuracy: 0.5106
Epoch 87/600
110/110 [==============================] - 4s 32ms/step - loss: 0.6915 - accuracy: 0.5137
Epoch 88/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6922 - accuracy: 0.5150
Epoch 89/600
110/110 [==============================] - 3s 27ms/step - loss: 0.6918 - 

110/110 [==============================] - 3s 31ms/step - loss: 0.6673 - accuracy: 0.5610
Epoch 160/600
110/110 [==============================] - 3s 32ms/step - loss: 0.6743 - accuracy: 0.5560
Epoch 161/600
110/110 [==============================] - 4s 39ms/step - loss: 0.6681 - accuracy: 0.5510
Epoch 162/600
110/110 [==============================] - 4s 34ms/step - loss: 0.6685 - accuracy: 0.5571
Epoch 163/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6686 - accuracy: 0.5571
Epoch 164/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6661 - accuracy: 0.5540
Epoch 165/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6655 - accuracy: 0.5603
Epoch 166/600
110/110 [==============================] - 3s 26ms/step - loss: 0.6672 - accuracy: 0.5534
Epoch 167/600
110/110 [==============================] - 3s 26ms/step - loss: 0.6691 - accuracy: 0.5573
Epoch 168/600
110/110 [==============================] - 3s 26ms/step - loss: 

110/110 [==============================] - 7s 63ms/step - loss: 0.6372 - accuracy: 0.5901
Epoch 238/600
110/110 [==============================] - 7s 60ms/step - loss: 0.6349 - accuracy: 0.6009
Epoch 239/600
110/110 [==============================] - 7s 67ms/step - loss: 0.6318 - accuracy: 0.5943
Epoch 240/600
110/110 [==============================] - 7s 68ms/step - loss: 0.6353 - accuracy: 0.6026
Epoch 241/600
110/110 [==============================] - 7s 64ms/step - loss: 0.6330 - accuracy: 0.6026
Epoch 242/600
110/110 [==============================] - 8s 74ms/step - loss: 0.6317 - accuracy: 0.6056
Epoch 243/600
110/110 [==============================] - 7s 63ms/step - loss: 0.6327 - accuracy: 0.6010
Epoch 244/600
110/110 [==============================] - 7s 60ms/step - loss: 0.6339 - accuracy: 0.5964
Epoch 245/600
110/110 [==============================] - 7s 60ms/step - loss: 0.6397 - accuracy: 0.5957
Epoch 246/600
110/110 [==============================] - 7s 65ms/step - loss: 

110/110 [==============================] - 7s 66ms/step - loss: 0.5803 - accuracy: 0.6454
Epoch 316/600
110/110 [==============================] - 7s 63ms/step - loss: 0.5893 - accuracy: 0.6450
Epoch 317/600
110/110 [==============================] - 7s 65ms/step - loss: 0.5833 - accuracy: 0.6471
Epoch 318/600
110/110 [==============================] - 7s 64ms/step - loss: 0.5859 - accuracy: 0.6453
Epoch 319/600
110/110 [==============================] - 7s 68ms/step - loss: 0.5758 - accuracy: 0.6614
Epoch 320/600
110/110 [==============================] - 7s 68ms/step - loss: 0.5838 - accuracy: 0.6476
Epoch 321/600
110/110 [==============================] - 9s 79ms/step - loss: 0.5884 - accuracy: 0.6474
Epoch 322/600
110/110 [==============================] - 7s 66ms/step - loss: 0.5848 - accuracy: 0.6421
Epoch 323/600
110/110 [==============================] - 7s 65ms/step - loss: 0.5960 - accuracy: 0.6444
Epoch 324/600
110/110 [==============================] - 8s 74ms/step - loss: 

110/110 [==============================] - 6s 57ms/step - loss: 0.5148 - accuracy: 0.7387
Epoch 394/600
110/110 [==============================] - 6s 55ms/step - loss: 0.5191 - accuracy: 0.7290
Epoch 395/600
110/110 [==============================] - 6s 54ms/step - loss: 0.5162 - accuracy: 0.7306
Epoch 396/600
110/110 [==============================] - 6s 55ms/step - loss: 0.5118 - accuracy: 0.7397
Epoch 397/600
110/110 [==============================] - 6s 54ms/step - loss: 0.5215 - accuracy: 0.7289
Epoch 398/600
110/110 [==============================] - 6s 53ms/step - loss: 0.5140 - accuracy: 0.7317
Epoch 399/600
110/110 [==============================] - 6s 51ms/step - loss: 0.5184 - accuracy: 0.7329
Epoch 400/600
110/110 [==============================] - 6s 55ms/step - loss: 0.5321 - accuracy: 0.7204
Epoch 401/600
110/110 [==============================] - 6s 55ms/step - loss: 0.5104 - accuracy: 0.7381
Epoch 402/600
110/110 [==============================] - 6s 55ms/step - loss: 

110/110 [==============================] - 7s 68ms/step - loss: 0.5054 - accuracy: 0.7349
Epoch 472/600
110/110 [==============================] - 7s 68ms/step - loss: 0.5331 - accuracy: 0.7217
Epoch 473/600
110/110 [==============================] - 7s 62ms/step - loss: 0.5068 - accuracy: 0.7349
Epoch 474/600
110/110 [==============================] - 7s 62ms/step - loss: 0.4971 - accuracy: 0.7401
Epoch 475/600
110/110 [==============================] - 6s 55ms/step - loss: 0.5061 - accuracy: 0.7390
Epoch 476/600
110/110 [==============================] - 7s 64ms/step - loss: 0.5076 - accuracy: 0.7363
Epoch 477/600
110/110 [==============================] - 7s 61ms/step - loss: 0.5092 - accuracy: 0.7346
Epoch 478/600
110/110 [==============================] - 7s 60ms/step - loss: 0.5090 - accuracy: 0.7351
Epoch 479/600
110/110 [==============================] - 7s 60ms/step - loss: 0.5052 - accuracy: 0.7389
Epoch 480/600
110/110 [==============================] - 7s 62ms/step - loss: 

110/110 [==============================] - 6s 56ms/step - loss: 0.5973 - accuracy: 0.6413
Epoch 550/600
110/110 [==============================] - 6s 59ms/step - loss: 0.5954 - accuracy: 0.6370
Epoch 551/600
110/110 [==============================] - 6s 57ms/step - loss: 0.5840 - accuracy: 0.6561
Epoch 552/600
110/110 [==============================] - 6s 58ms/step - loss: 0.5599 - accuracy: 0.6774
Epoch 553/600
110/110 [==============================] - 6s 55ms/step - loss: 0.5728 - accuracy: 0.6633
Epoch 554/600
110/110 [==============================] - 6s 57ms/step - loss: 0.5618 - accuracy: 0.6764
Epoch 555/600
110/110 [==============================] - 6s 58ms/step - loss: 0.5510 - accuracy: 0.6893
Epoch 556/600
110/110 [==============================] - 6s 54ms/step - loss: 0.5454 - accuracy: 0.7011
Epoch 557/600
110/110 [==============================] - 6s 57ms/step - loss: 0.5376 - accuracy: 0.7041
Epoch 558/600
110/110 [==============================] - 6s 56ms/step - loss: 

[1.1290781497955322, 0.550000011920929]